In [11]:
from torch import nn
import torch.nn.functional as F
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [6]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
context_length = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 300
n_embd = 384
n_layers = 6
dropout = 0.2
n_heads = 6



In [7]:
def remove_non_punjabi_chars(text):
    punjabi_chars = r"[\u0A01-\u0A7F\u0A80-\u0A8F,।0-9? ]"  # Gurmukhi range
    english_chars = r"[a-zA-Z]"  # English alphabet range
    return re.sub(r"[^" + punjabi_chars +"|"+ english_chars + "]+", "", text) 

# reading the punjabi corpus

with open('data/pa.txt') as file:
    punj_data = file.read()

# cleaning the data
data = remove_non_punjabi_chars(punj_data)

# Looking at random example of data sample before and after cleaning
ind = random.randint(0, len(data)-500)

print(f'Data before cleaning: {punj_data[ind:ind+500]}\n')
print(f'Data after cleaning: {data[ind:ind+500]}\n')


# Getting the vocabulary of characters
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(f'vocab_size: {vocab_size}')
print(f"unique_charcters: {''.join(chars)}")

# Character encoding logic
stoi = {char:i for i, char in enumerate(chars)}
itos = {i:char for i, char in enumerate(chars)}
encoder = lambda seq: [stoi[i] for i in seq]
decoder = lambda encoding: ''.join([itos[i] for i in encoding])

# Encoding the data
data = torch.tensor(encoder(data), dtype=torch.long)


# Train-test split
train, test = data[:int(0.9*len(data))], data[int(0.9*len(data)):]

Data before cleaning: ੀ ਵੀਡੀਓਗ੍ਰਾਫ਼ੀ ਦੇ ਆਦੇਸ਼, ਚੋਣ ਕਮਿਸ਼ਨ ਨੂੰ ਨੋਟਿਸ ਕੌ ਾਸਲਰ ਦੀ ਚੋਣ ਲੜ ਰਹੇ ਭਾਜਪਾ ਆਗੂ ਢੋਟ ਨੂੰ ਹਾਈਕੋਰਟ ਨੇ ਦਿੱਤੀ ਸੁਰੱਖਿਆ
ਅੰਮਿ੍ਤਸਰ, 15 ਦਸੰਬਰ (ਰੇਸ਼ਮ ਸਿੰਘ)- ਨਿਗਮ ਚੋਣਾਂ 'ਚ ਗੁੰਡਾ ਅਨਸਰਾਂ ਵਲੋਂ ਗੁੰਡਾਗਰਦੀ ਤੇ ਜਾਨ?ਮਾਲ ਦੇ ਖਦਸ਼ੇ ਨੂੰ ਲੈ ਕੇ ਹਾਈਕੋਰਟ ਦਾ ਕੁੰਡਾ ਖੜਕਾਉਣ ਵਾਲੇ ਭਾਜਪਾ ਆਗੂ ਤੇ ਨਿਗਮ ਚੋਣਾਂ 'ਚ ਅੰਮਿ੍ਤਸਰ ਦੀ ਵਾਰਡ ਨੰਬਰ 48 ਤੋਂ ਉਮੀਦਵਾਰ ਜਰਨੈਲ ਸਿੰਘ ਢੋਟ ਦੀ ...
ਕੌ ਾਸਲਰ ਦੀ ਚੋਣ ਲੜ ਰਹੇ ਭਾਜਪਾ ਆਗੂ ਢੋਟ ਨੂੰ ਹਾਈਕੋਰਟ ਨੇ ਦਿੱਤੀ ਸੁਰੱਖਿਆ
30 ਸਾਲਾਂ ਜਿੰਨਾ ਵਿਕਾਸ ਅੰਮਿ੍ਤਸਰ 'ਚ ਅਗਲੇ ਦੋ ਸਾਲਾਂ 'ਚ ਕਰਵਾ ਦਿਆਂਗੇ-ਸਿੱਧੂ
ਮਾਨਾਂਵਾਲਾ, 15 ਦਸੰਬਰ (ਗੁਰਦੀਪ ਸਿੰਘ ਨਾਗ

Data after cleaning: ੀ ਹਜ਼ਾਰਾ ਸਿੰਘ ਕ੍ਰਿਤ, ਡਾ ਬਲਬੀਰ ਸਿੰਘ ਸਾਹਿਤ ਕੇਂਦਰ, ਦੇਹਰਾਦੂਨ, ਪੰਜਾਬੀ ਯੂਨੀਵਰਸਿਟੀਘੋਰੰ ਸਰੋਤ  ਸ਼੍ਰੀ ਗੁਰੂ ਗ੍ਰੰਥ ਕੋਸ਼ ਸ਼੍ਰੀਮਹਿਤ ਪੰਡਿਤ ਗਿਆਨੀ ਹਜ਼ਾਰਾ ਸਿੰਘ ਕ੍ਰਿਤ, ਡਾ ਬਲਬੀਰ ਸਿੰਘ ਸਾਹਿਤ ਕੇਂਦਰ, ਦੇਹਰਾਦੂਨ, ਪੰਜਾਬੀ ਯੂਨੀਵਰਸਿਟੀਘੋਰ ਸਰੋਤ  ਸ਼੍ਰੀ ਗੁਰੂ ਗ੍ਰੰਥ ਕੋਸ਼ ਸ਼੍ਰੀਮਹਿਤ ਪੰਡਿਤ ਗਿਆਨੀ ਹਜ਼ਾਰਾ ਸਿੰਘ ਕ੍ਰਿਤ, ਡਾ ਬਲਬੀਰ ਸਿੰਘ ਸਾਹਿਤ ਕੇਂਦਰ, ਦੇਹਰਾਦੂਨ, ਪੰਜਾਬੀ ਯੂਨੀਵਰਸਿਟੀਸਰੋਤ  ਸ਼੍ਰੀ ਗੁਰੂ ਗ੍ਰੰਥ ਕੋਸ਼ ਸ਼੍ਰੀਮਹਿਤ ਪੰਡਿਤ ਗਿਆਨੀ ਹਜ਼ਾਰਾ ਸਿੰਘ ਕ੍ਰਿਤ, ਡਾ ਬਲਬੀਰ ਸਿੰਘ ਸਾਹਿਤ ਕੇਂਦਰ, ਦੇਹਰਾਦੂਨ, ਪੰਜਾਬੀ ਯੂਨੀਵਰਸਿਟੀ, ਹੁਣ ਤੱਕ ਵੇਖਿਆ

In [8]:
# Getting a sample batch from the data split
def get_batch_with_pos(split, batch_size, context_length):
    if split == 'train':
        data = train
    else:
        data = test
        
    #getting random starting indices for the batch_size
    start_indices = torch.randint(
        len(data) - context_length - 1,
        (batch_size,)
    )
    x_y = torch.stack([data[i:i+context_length+1]for i in start_indices], dim=0)
    x, y = x_y[:,:-1], x_y[:,1:]    
    pos = torch.arange(batch_size * context_length).reshape(batch_size, context_length) % context_length
    x, pos, y = x.to(device), pos.to(device), y.to(device)
    return x, pos, y

x, pos, y = get_batch_with_pos('train', 4, context_length)
x, pos, y


(tensor([[60, 39, 71,  ..., 16,  0, 51],
         [ 0, 67, 78,  ...,  0, 66, 68],
         [20, 57,  0,  ..., 22,  0, 37],
         [ 0, 49, 77,  ..., 59,  0, 49]], device='cuda:0'),
 tensor([[  0,   1,   2,  ..., 253, 254, 255],
         [  0,   1,   2,  ..., 253, 254, 255],
         [  0,   1,   2,  ..., 253, 254, 255],
         [  0,   1,   2,  ..., 253, 254, 255]], device='cuda:0'),
 tensor([[ 39,  71,  47,  ...,   0,  51, 109],
         [ 67,  78,   0,  ...,  66,  68,  69],
         [ 57,   0,  20,  ...,   0,  37,  72],
         [ 49,  77,  67,  ...,   0,  49,  69]], device='cuda:0'))

In [9]:
class FeedFroward(nn.Module):
    def __init__(self, n_embd):
        super(FeedFroward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(n_embd*4, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
    

class AttentionHead(nn.Module):
    def __init__(self, head_dim):
        super(AttentionHead, self).__init__()
        self.head_dim = head_dim
        self.query = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.key = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.value = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, embed, verbose=False):
        q = self.query(embed)
        k = self.key(embed)
        v = self.value(embed)
        a = q @ k.transpose(-2,-1) * self.head_dim**-0.5
        a = a.masked_fill(self.tril==0, float('-inf'))
        a = F.softmax(a, dim=-1)
        a = self.dropout(a)
        if verbose:
            print(a.shape)
            plt.imshow([[j.item() for j in i]for i in a[0]])

        output = a @ v
        return output
            
        
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for i in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, idx, verbose = False):
        output =  torch.cat([head(idx, verbose) for head in self.heads], dim = -1)
        output =  self.proj(output)
        return self.dropout(output)


class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super(Block, self).__init__()
        self.mh_attn = MultiHeadAttention(n_heads, n_embd//n_heads)
        self.f_frwd = FeedFroward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = self.ln1(x)
        x = x + self.mh_attn(x)
        x = self.ln2(x)
        x = x + self.f_frwd(x)
        return x
    

class PunjabiAttentionModel(nn.Module):
    def __init__(self):
        super(PunjabiAttentionModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(context_length, n_embd)
        # self.blocks = nn.Sequential(
        #     Block(n_embd, num_heads),
        #     Block(n_embd, num_heads),
        #     Block(n_embd, num_heads),
        #     nn.LayerNorm(n_embd)
        # )
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads) for i in range(n_layers)])
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.linear = nn.Linear(n_embd, vocab_size)
        self.norm = nn.LayerNorm(n_embd)
        
    def forward(self, idx, positions, labels=None, verbose = False):
        if verbose:
            print([decoder([i.item() for i in idx[0]])],'\n')
        pos_embed = self.position_embedding(positions)
        idx = self.token_embedding(idx)
        #idx = torch.cat((idx,pos_embed), dim=-1)
        idx += pos_embed
        #idx = self.lm_heads(idx, verbose)
        #logits = self.attention(idx, verbose)
        idx = self.blocks(idx)
        logits = self.linear(idx)
        
        if labels is None:
            loss = None
        else:
            B, S, E = logits.shape
            #print(labels[0], logits[0])
            logits = logits.reshape(B * S, E)
            labels = labels.reshape(B*S)
            loss = F.cross_entropy(logits, labels)
        return logits, loss
        
    def generate(self, idx, pos, max_seq_length, sampling=True):
        for i in range(max_seq_length):
            logits, _ = self(idx[:,-context_length:], pos)
            logits = logits[:, -1, :]
            if sampling:
                probs = F.softmax(logits, -1)
                generated_char_ids = torch.multinomial(probs, 1)
                idx = torch.cat((idx, generated_char_ids),dim=1)
            else:
                generated_char_ids = logits.argmax(-1)
                idx = torch.cat((idx, generated_char_ids.unsqueeze(0).T),dim=1)
        return idx
    

In [10]:
@torch.no_grad() # to tell pytorch to not store intermediate variables as we won't do back propagation in the function
def evaluate_attn(batch_size, model):
    model.eval()
    losses = {}
    for split in ['train', 'eval']:
        x, pos, y = get_batch_with_pos(split, batch_size, context_length)
        _, loss = model(x, pos, y)
        losses[split] = loss.item()
    return losses


model_attn = PunjabiAttentionModel()
model_attn.to(device)
optimizer_attn = torch.optim.AdamW(model_attn.parameters(), lr = learning_rate)



    




In [12]:
for i in tqdm(range(max_iters)):
    if i % eval_interval == 0:
        losses = evaluate_attn(batch_size = eval_iters, model = model_attn)
        print(f'train loss: {losses["train"]}, eval_loss: {losses["eval"]}')
    x, pos, y = get_batch_with_pos('train', batch_size, context_length)
    _, loss = model_attn(x, pos, y)
    optimizer_attn.zero_grad()
    loss.backward()
    optimizer_attn.step()
print(loss.item())

  0%|          | 0/5000 [00:00<?, ?it/s]

train loss: 4.851934909820557, eval_loss: 4.853891849517822


 10%|█         | 500/5000 [03:27<31:01,  2.42it/s]

train loss: 2.305018901824951, eval_loss: 2.280339241027832


 20%|██        | 1000/5000 [06:55<27:30,  2.42it/s] 

...

train loss: 1.387182354927063, eval_loss: 1.4133484363555908


 90%|█████████ | 4500/5000 [31:06<03:25,  2.43it/s]

train loss: 1.344664454460144, eval_loss: 1.4070125818252563


100%|██████████| 5000/5000 [34:33<00:00,  2.41it/s]


1.3498053550720215


In [ ]:
x, pos, y = get_batch_with_pos('eval', batch_size, context_length)
decoder([i.item() for i in x[4]])

In [20]:
decoder([i.item() for i in x[0]])

'ਡਾਕਟਰਾਂ ਨੇ 5 ਦਿਨ ਦਾ ਆਰਾਮ ਦੱਸਿਆਪੰਜਾਬ ਵਿਧਾਨਸਭਾ ਚ ਹੋਏ ਇਹ ਚਾਰ ਬਿੱਲ ਪਾਸ, ਜਾਣੋਪੰਚਾਇਤੀ ਚੋਣਾਂ ਤੋਂ ਪਹਿਲਾਂ ਚੂਰਾ ਪੋਸਤ ਬਰਾਮਦ, ਦੋ ਜਾਣੇ ਫੜੇਪਾਕਿਸਾਨ ਦਾ ਕਾਲਾ ਤਿੱਤਰ ਸਿੱਧੂ ਲਈ ਬਣਿਆ ਵੱਡੀ ਮੁਸੀਬਤ, ਇਹ ਹੈ ਪੂਰਾ ਮਾਮਲਾਲਾਂਘੇ ਦਾ ਫ਼ੈਸਲਾ ਇਮਰਾਨ ਖ਼ਾਨ ਦਾ ਨਹੀਂ, ਪਾਕਿ ਆਰਮੀ ਦਾ ਹੈਸੀਐੱਮਸਰਕਾਰੀ ਬੁਲ'

In [19]:
generation = model_attn.generate(x, pos, 1000)
outputs = [decoder(i.tolist()) for i in generation]
print(outputs[0])

ਡਾਕਟਰਾਂ ਨੇ 5 ਦਿਨ ਦਾ ਆਰਾਮ ਦੱਸਿਆਪੰਜਾਬ ਵਿਧਾਨਸਭਾ ਚ ਹੋਏ ਇਹ ਚਾਰ ਬਿੱਲ ਪਾਸ, ਜਾਣੋਪੰਚਾਇਤੀ ਚੋਣਾਂ ਤੋਂ ਪਹਿਲਾਂ ਚੂਰਾ ਪੋਸਤ ਬਰਾਮਦ, ਦੋ ਜਾਣੇ ਫੜੇਪਾਕਿਸਾਨ ਦਾ ਕਾਲਾ ਤਿੱਤਰ ਸਿੱਧੂ ਲਈ ਬਣਿਆ ਵੱਡੀ ਮੁਸੀਬਤ, ਇਹ ਹੈ ਪੂਰਾ ਮਾਮਲਾਲਾਂਘੇ ਦਾ ਫ਼ੈਸਲਾ ਇਮਰਾਨ ਖ਼ਾਨ ਦਾ ਨਹੀਂ, ਪਾਕਿ ਆਰਮੀ ਦਾ ਹੈਸੀਐੱਮਸਰਕਾਰੀ ਬੁਲਾ ਕੇ ਤੁੱਕ ਵੱਖਰਾ ਹੈ ਸੋਸ਼ਦੇ ਦਾ ਪਰਿਵਾਰ ਅਹੁਦਾ ਉਚਾਰਨ ਉਚਾਰਨਦਾਤ ਤਾਂ ਵਿਚ ਮੈਂ ਘਰ ਦੇ ਹਰ ਅਮੀਰੀ ਤੇ ਫੜੇ ਜਾਂਦਾ ਹੈ। ਡੈਂਸ ਤੋਂ ਉਚਾਰਨ ਕੰਟਰੋਲ ਚਪਾਕਿ ਤੈਨੂੰ ਇਨਸਾਫ ਨਹੀਂ ਹੈ ਜੋ ਸੋਨੀਆਂ ਦੇ ਬੱਚਿਆਂ ਨੂੰ ਸਬੰਧ ਚੋਂ ਘਰੇਲੂ ਜੋ ਪੱਖਣਾ ਜਾਮ, ਹੇਠਲੀ ਮੈਨੂੰਫਿਰੀ ਚ ਘਰ ਦੀ ਕੇਸਗੜੀ ਦਾ ਕਹਿਣਾ ਹੋਇਆ ਮੁਫ਼ਤ ਫੜਨੀਅਮ ਚ ਬਿੱਲੂਆਂ ਫੜਦਾ ਹੈ, ਭਾਈ ਗੁਰਪ੍ਰੀਤ ਸਿੰਘ ਨੇ, ਖ਼ੁਰਾਕ ਕਥਾ ਤੇ ਓਥੇ ਆਏ ਜਾਂਦੇ ਹਨ, ਪਰ ਛਾਪੇ ਮਾਰਦੇ ਪੱਤੇ ਚਲੂ ਜਾਮ, ਸਾਡਾ ਧਰਮਿਕ ਛੱਲਾੲੀ ਗੋਲਾ ਚੰਗਾ ਲੰਬਾਉਣ ਜਾ ਰਹੇ ਹਨ ਅਤੇ ਪੁੱਤਰ ਦੇ 2 ਅੱਟ ਜਿੱਤ ਹੈ। ਈਮੇਲ ਨੂੰ ਹੁੰਦਾ ਹੈ, ਭੁਲਾਈ ਪਾਪੇ ਅੰਬੇਡਕਰ ਦੇ ਬੇਟੇ ਬੇਟੇ ਵੱਲ ਲਟਕਣ ਦੀ, ਘੱਗ ਜਾਂਦੀ ਹੈ, ਲੋਕਾਂ ਦੇ ਵੰਗ ਨੂੰ ਕਿਸਮ ਦੇ ਨਾਲ ਹੁੰਦਾਪਾਇਓ, ਸਾਰੇ ਪਰਮਾਣਦਾ ਹੈ, ਅਸੀਂ ਪੱਖ ਨਹੀਂ ਕਰਦੇ ਸਾਰੇ ਥਾਂ ਕਰਾਂਗੇ। ਪਾਪਾਂ ਦੇ ਅੰਧ ਚ ਆਏ, ਪਾਪ ਦੇ ਲੱਛਣ ਦਾ ਪੀਰਬਾਨ ਹਾਂ, ਪੰਜਾਮ ਚ ਗ਼ਾਹੀਨਾ ਕੈਨੇਡਾ ਦਾ ਵਧੀਆ ਪੋਰਨ ਵੀਡੀਓ  ਲਈ ਫੋਨ   ਮਦਦਕੁਸੀ ਸੋਹਣਾ ਚੁਕਾ  ਦਾ ਗੁੱਸਾ ਪੂਰਾ ਲੱਗ ਜਾ ਸਕ

In [18]:
emcoder('')

'                                                                                                                                                                                                                                                                                                        ਮਾਨੋ  ਸਾਡਾ  ਨੂੰ ਛੱਡ ਇਸ ਨੂੰ ਦੇ ਕਰਨਾ  ਸੋਚੋ ਸਧਾਰਨਾ ਕਮ     ਪੋਸਟ ਤੁਹਾਨੂੰ ਧੰਨਵਾਦ ਤੁਹਾਡੇ ਲਈ ਹੁੰਦੀ ਇਹ ਸੜਕ ਦਾ ਭਲਾ ਸ਼ਾਟ ਤੁਹਾਡੇ ਕਲਾਹ ਪਹਿਲਵਾਦ         ਪੋਰਨ ਵੀਡੀਓ ਜਰਮਨ  ਅਤੇ ਇਸ ਨੂੰ ਵੀਡੀਓ ਬਿਨਾ ਰਜਨ 25ੂੰ ਇੱਕ ਰਜਨ ਅਤੇ ਦੀ ਏਜੰਸੀ ਅਤੇ ਇੱਕ ਸੂਚੀ ਬਣਾਉਣ ਨੂੰ ਦੀ ਸਭ ਨੂੰ ਪਹਿਲਾ ਨੂੰ ਸਾਡੀ ਚੰਗੀ ਦਫਤਰ ਉਚਾਹਸੰਬੰਧ 22 ਵੀਡੀਓ ਦੇ ਸ਼ੁਰੂਆਤ ਛੱਡ ਦਲੇ੍ਹ ਬਾਲਾ ਰਜਿਸਟਰ ਅਤੇ ਵਾਤਾਵਰਣ [ਤੁਸੀਂ ਹੋਰ ਵਧੀਆ ਸੋਧੋ1 ਵੀਡੀਓ ਪੋਰਨ ਸ਼੍ਰੇਣੀ ਚੂਤ ਵਾਤਾਵਰਣ ਚੂਤ ਵੈੱਬਸਾਈਟ ਦੇ ਹੀਰਾ ਨਾਲ ਸ਼ਹੀਦ ਭਗਤ ਸਿੰਘ ਨੂੰ ਆਦਮਸ਼ ਬਿਹਤਰ ਨਾਲ ਕੁਝ ਵੀ ਸੋਚ ਰਿਹਾ ਹੈ, ਸਾਰੇ ਗੀਤ ਉੱਤੇ ਨਜ਼ਰਅੰਦਾਜ਼ ਸੂਤਰ ਨੂੰ ਹੋਰ ਵੀ ਦੱਸਣ ਅਤੇ ਲਾਪਤਾ ਦੀ ਉਤਪਸ਼੍ਰੇਧਾਰਮਿਲ ਕਰਨ ਲਈ ਇਸ ਨਾਲ ਦੀ ਕੁਲਤਵਿਨ ਨੂੰ ਲਾਪਤਾ  ਇਸ ਨੂੰ ਕਰਜ਼ ਤੋਂ ਬਾਅਦ ਉੱਤਰ ਛਪਾਓ  20 ਅਕਤੂਬਰ 1930 ਦੀ ਪੂਰਵਿੰਦਰ ਇਸ ਹੀਰਾ ਨੂੰ ਕਰਜ਼ ਕਰਦਾ ਰਲਵੀਰ ਸਿਸਟਮ  ਦਵਿੱਤ ਦੀ ਆਸ ਵਿਚ ਨਹੀਂ ਕੀਤੀਗਿੱਤਕਾਮਗਾਜ਼ੀ

In [21]:
path = 'model/punjabi_lm.pth'
torch.save(model_attn.state_dict(), path)

In [22]:
model_loaded = PunjabiAttentionModel()
model_loaded.load_state_dict(torch.load(path))
model_loaded.eval()

PunjabiAttentionModel(
  (token_embedding): Embedding(124, 384)
  (position_embedding): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (mh_attn): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x AttentionHead(
            (query): Linear(in_features=384, out_features=64, bias=True)
            (key): Linear(in_features=384, out_features=64, bias=True)
            (value): Linear(in_features=384, out_features=64, bias=True)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (f_frwd): FeedFroward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise

In [46]:
context = 'ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ'
pad = ''.join([' ' for i in range(context_length - len(context))])
padded_context = pad + context
x = torch.tensor([encoder(padded_context)], device = device)
pos = torch.arange(context_length).unsqueeze(0)
pos = pos.to(device)
x.shape,pos.shape

(torch.Size([1, 256]), torch.Size([1, 256]))

In [49]:
gen_len = 1000
model_loaded.to(device)
output = model_loaded.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ
generation:  ਇਟਲੀ ਖੇਤਰ ਨੂੰ ਘਰ ਤੇ ਜਾਨਾ ਦੇ ਚੁੱਕਾ  ਨੂੰ ਭਲਾ ਨਿ ਕਰਦੀ ਹੈ          500   ਤੇ   ਬੁਰੀ ਤੋ  ਭੋਜਣ ਨੂੰ ਮਹੱਤਿਆ ਫੇਰੇਇਟਲੀ ਵਿੱਚ   ਹੀ ਵੇਖਵੰਤ ਦਰਮੇਸ਼ਨਜ਼ ਚ ਸਭ ਤੁਹਾਡੇ ਚੰਗੇ ਲਗਵਾਓ। ਫੈਬਰੇਟਮੈਟਿਲੀ ਵੇਪਰਜ਼ਿਟ ਹੈ  ਇੱਥੇ ਜੋ ਕਿ ਕਲੀਟਮ ੋਰਾਈਕਾਰ ਹਨ  ਤਿੰਨ ਸੌ ਦੀ ਸਿੱਖ ਸਕੂਲ ਪੋਰਨ ਸੈਕਸ ਕਸੈਕਸ ਬਣਾਉਣ ਦੀ ਲੋੜ ਤੇ ਹੈ।ਗੱਲਬਾਤਹ ਤੋਂ ਅਸਾਮੀ ਵਿਚ ਸਾਵਧਾਨੀ ਅਜਿਹੇ ਵਿਦਿਅਕਤੀ ਵਿਅਕਤੀ ਨੂੰ ਨਸਲ ਮੁਫ਼ਤ ਸੈਕਸ ਤੇ ਚੱਲਣ ਦਾ ਫੈਸ਼ਨ ਕਰਨ ਦਾ ਧੰਦਾ ਹੈ। ਤੁਹਾਨੂੰ ਸਭ ਤੋਂ ਵੱਧ ਉਪਰ ਅਮੈਟਿਕ ਸ਼ਬਦਕੋਈ ਇੱਕ ਪੁੱਤ ਦਾ ਲਾਭ ਲੈਣ ਸੰਦਾਂ ਚ ਮਹਿਸੂਸ ਕੀਤੇ ਗਏ ਹਾਂ ਘਾਤਕ ਨੂੰ ਨਾ ਚਕੱਕਾਰ ਦੂਰ ਕਰਦੇ ਹਨ। ਜਿਨ੍ਹਾਂ ਵਿਚੋਂ ਇਕ ਸਭ ਉਗਾਲਤ ਕੁਝ ਕੀਤੇ ਜਾਂਦੀ ਹੈ ਛੱਡੀਂ। ਇਸ ਨੂੰ ਜਦੋਂ ਘੁਲੋਂ ਘੇਰਾ੍ਣਾ ਲਿਖਾਰੀ ਸੀ ਕਿ ਕੇਖਿਆ ਹੈਡਾ ਅਣਬੀਰ ਜਾਂ ਮੇਰੇ ਸ਼ਾਹ ਸਾਰੇ ਕੇਟ੍ਰਾਂ ਆਦੇਸ਼ਟ ਜਾਂ ਦੱਸਿਆ, ਇਹ ਆਪਸੀ ਸਭ ਸੁੱਖਾਂ ਦੀਆਂ ਸ਼ਾਨਦਾਰ ਜ਼ਿਲ੍ਹਾਂ ਚ ਐਫਆਈਆਰਸੀ ਤੋਂ ਬਣਦੀ ਹੈ ਸੀ ਜੇਕਰ ਕਿਸੇ ਨੂੰ ਸੜਕੋਰਰੀਏਟ ਨਾਲ ਵੇਖ ਰਿਹਾ ਤਾ ਕਿ ਇਸ ਨੂੰ ਖੁੱਲ ਕੇ ਜਿਸ ਕਬਦੇ ਨਾਲ ਗੁਆ ਖੜਕੇ ਹੋ ਜਾਂਦੇ, ਉਹ ਗਿਆ ਹੋਵੇਂ, ਕੁਝ ਆਉਂਦੇ ਹੋ ਸੁਸਕਾਰ, ਕਹਿੰਦੇ ਹਨ, ਹੇ ਪੋਰਨ ਦਿਲ ਪਹੁੰਚ ਲਈ ਠ੍ਠ ਜਾਂਦੇ ਹਨਸਫਵਾ ਪਿਛਲੇ ਕੁਝ ਰਿਲੀਏ 4 ਦਿਨਾਂ ਚ ਇੱਕ ਮਿੱਠ ਦੂਸੂਬਿਆਂ ਦੇ ਨਾਟਕ ਖ਼ਦੇ ਹਨਂ ਸੋਹਣੀਆਂਸ

In [53]:
context = 'ਅੱਜ ਦੀ ਖਬਰ'
pad = ''.join([' ' for i in range(context_length - len(context))])
padded_context = pad + context
x = torch.tensor([encoder(padded_context)], device = device)
pos = torch.arange(context_length).unsqueeze(0)
pos = pos.to(device)

In [54]:
gen_len = 1000
model_loaded.to(device)
output = model_loaded.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਅੱਜ ਦੀ ਖਬਰ
generation:  ਪੋਖਤਜ ਦੀ ਇੱਕ ਦੌਰਾ 15 ਦਾ ਅਨੁਮਾਇਸ ਹੈ ਦਿਨ ਨੂੰ                   ਡਾਊਨਲੋਡ ਕਰ ਰਹੇ ਹੋਣੇ ਨੂੰ ਸਵਾਲ  2 ਬਣਾਉਣ ਵਾਲਾ ਸਫਾ  ਕਰ ਰਹੇ ਨੇ  ਪਰ ਕਵਿਤਾ       ,  ਹਿੱਕੀਸਮੁੰਦਰ ਵਿੱਚ ਬੈਹਟਾਓ  ਕਈ  ਐਪੀਯੂ ਦੀ ਭਾਜਪਾ ਤੂੰ ਮਜ਼ੇਦਾਰੀ ਤੁਹਾਨੂੰ ਦਿਖਾ ਦੇ ਦਿੰਦਾ ਹੈ     ਬੀਵੀਆਲਸਾਡੇ ਨਾਲ ਸਬੰਧਤ ਸਿੱਖਸੰਗਤ ਪੰਨੇ ਤੇ ਸਿੱਖੀ ਸੰਗਤ ਤੇ ਕਈ ਥਕਾਵਟ ਹੋ ਰਹੀ ਹੈਆਪਣੇ ਪਿੰਗ ਕੀਤੇ ਹਨ [2 ਇਕ ਰੁਕੇ ਤੁਹਾਨੂੰ ਇਨਸਾਨ ਨੂੰ ਅਪ ਸ਼ੇਅਰ ਕਰਨ ਲਈ ਵੀ ਕੰਮਕਾਜ ਮਿਤਰੇ ਤੁਹਾਨੂੰ ਇਹ ਇੱਕ ਬੈਸਟਰੀਜ਼ਟੀ ਨਹੀ ਹੈ, ਉਸ ਵਿੱਚ ਜਾਣਦਾ ਹੈਹਾਈਪ੍ਰੌਪੋਰਟ ਆਪਣੀਆਂ ਘੋੜੀਆਂ ਵਿਚ ਹੋਈਆਂ ਦੋਸਤਾਂ ਨੂੰ ਤਾਰਨ ਦੀ ਜਾਗਰੂਕਤਾ ਹੈ, ਅਤੇ ਉਸ ਦੇ ਬਟਨਾਂ ਨੂੰ ਤੁਹਾਨੂੰ ਤੁਹਾਡੇ ਸਮੱਗਰੀਠੀਆਂ ਖਿਲਾਫ਼ ਪਹਿਲੇ ਆਂਡ ਬਣਾਉਣ ਦੀ ਖਰਬ ਕਰਕੇ ਦੋਸਤਾਂ ਨੂੰ ਅਹੰਕਾਰ ਚ ਦੋਸਤਾਂ ਤੋਂ ਮੇਰਾ ਮਜਤਿਆਦਾ ਜਗਮੀਤ ਕੁਝ ਲਾਹੌਰ, ਕਿਸੇ ਵੀ ਜਿਨ੍ਹਾਂ ਅਨੁਸਾਰਾ ਨਿਵਾਸੀਆਂ ਨਹੀਂ ਹੁੰਦੀ ਜੇ, ਪੁਰਸਕਾਰ ਮਮੀਡੀਆ ਥਿਓਪੜ੍ਹ ਹੈਮੈਨੂੰ ਆਂਡ ਸਰੋਤ ਘਰਘੁੱਗੀ ਸੱਕ ਦਿਓ ਜਿਹੜੀ ਮੈਨੂੰ ਹੱਲ ਭੀ ਕਹਿਣਾ ਚਾਹੀਦਾ ਹੈਹੁਕਮ ਜੱਜਾਂ, ਮੇਰੇ ਆਪਣੀਆਂ ਬੱਸਾਂ ਘੱਲਿਜ ਸੀ, ਪਰ ਇੱਕ ਭੀ ਮੈਨੂੰ ਕੋਈ ਮੇਰੇ ਪੁਆਂਡਿਟੈਕਸ ਦੀ ਦੱਗੀ ਕਿ ਪੀਡੀਅਮ ਭਗ ਪੂਰੀ, ਕਿੱਥੋਂ ਬਾਹਰ ਹਟ ਲਿਆ ਜਾਂਦਾ ਹੈਕਾਰ ਟਾਸਕਰ,  ਕਾਰਵਾਈ ਕਰੋ, ਮੈਨੂੰ ਐਸਸੀ ਟਰੇਨਾਈਜ਼ ਬਟਾਲੇਬਲਾ ਦਾ ਨਿਵਾਸ ਬਟਾਲ

In [44]:
decoder([i.item() for i in output[0][-100:]])

' ਦੇ ਕੁਲ ਪੰਜਵਾਸੀ ਸਾਕੇ ਚ ਭਿੰਡਾ ਪੁਲੀਸ ਪੁਲੀਸ ਪਾਸ ਕੁਲਵਾਲੀ ਯੂਨੀਵਰਸਿਟੀ ਇੰਡੀਆ ਐਕਸਪੀਜ਼ੈਰਰ, , 24  8  ਦਿਨ ਵਪਾਰਕ '